# Imports

In [1]:
import io
import pathlib
import pickle
import re

import ads
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
import galstreams
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from astropy.stats import median_absolute_deviation as MAD
from gala.units import galactic
from helpers import (
    get_default_track_for_stream,
    get_frame_from_points,
    get_full_galstreams_poly,
    get_isochrone,
    make_ibata_poly_nodes,
    run_orbit_fit,
)
from pyia import GaiaData
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.optimize import minimize
from tqdm.auto import tqdm

%matplotlib inline

Fiducial isochrone model:

In [2]:
iso = get_isochrone()

tmp_iso = iso[iso["stage"] == 1]
MSTO_absmag = tmp_iso["rP1"].min()

In [3]:
mwstreams = galstreams.MWStreams()

Initializing galstreams library from master_log... 


# Load data

## Ibata:

In [4]:
# tbl = at.Table.read("../data/Ibata2023_Table1.csv")
ibata_g = GaiaData(
    "../data/Ibata2023_GaiaDR3-xm.fits",
    radial_velocity_colname="vh",
    radial_velocity_unit=u.km / u.s,
)

In [5]:
ibata_tbl = at.Table.read("../data/ibata_streams_full.csv")
sID_to_name = {row["sID"]: row["name"] for row in at.unique(ibata_tbl, keys="sID")}
name_to_sID = {v: k for k, v in sID_to_name.items()}

In [6]:
_dist = ibata_g.get_distance(allow_negative=True)
_dist[~np.isfinite(_dist)] = 100 * u.Mpc  # put them very far away if missing

ibata_c = ibata_g.get_skycoord(distance=_dist)

## S5 streams:
s5_streams = "300S",\n"Willka Yaku",\n"AAU", "Jet", "Phoenix", "Indus", "Palca", "Elqui", "Turranburra",

Distance modulus functions are from Li et al 2022, Table 1

In [7]:
rows = []

In [8]:
stream = mwstreams["Orphan-K19"]

orp_sky_tbl = at.Table.read(
    "../data/orphan/koposov2023-measurement_tables/fitses/orphan_M_track_bins.fits"
)
orp_phi1_grid = np.linspace(orp_sky_tbl["phi1"].min(), orp_sky_tbl["phi1"].max(), 1024)
_spl = InterpolatedUnivariateSpline(orp_sky_tbl["phi1"], orp_sky_tbl["phi2"], k=3)
orp_phi2_grid = _spl(orp_phi1_grid)

orp_dm_tbl = at.Table.read(
    "../data/orphan/koposov2023-measurement_tables/fitses/orphan_dmrr_bins.fits"
)
_spl = InterpolatedUnivariateSpline(orp_dm_tbl["phi1"], orp_dm_tbl["dm"], k=3)
orp_dm_grid = _spl(orp_phi1_grid)
track = coord.SkyCoord(
    phi1=orp_phi1_grid * u.deg,
    phi2=orp_phi2_grid * u.deg,
    distance=coord.Distance(distmod=orp_dm_grid),
    frame=stream.stream_frame.replicate_without_data(),
)

orp_width_tbl = at.Table.read(
    "../data/orphan/koposov2023-measurement_tables/fitses/orphan_M_width_bins.fits"
)
_spl = InterpolatedUnivariateSpline(
    orp_width_tbl["phi1"], orp_width_tbl["log_width"], k=3
)
orp_width_grid = np.exp(_spl(orp_phi1_grid))
sky_width = np.mean(orp_width_grid) * u.deg
width_pc = np.mean(orp_width_grid * u.deg * track.distance).to(
    u.pc, u.dimensionless_angles()
)

rows.append(
    {
        "name": "OC",
        "gs_track": "Orphan-K19",
        "other_names": ["Orphan", "Chenab"],
        "phi1": track.phi1.wrap_at(180 * u.deg),
        "phi2": track.phi2,
        "sky_track": track,
        "sky_ref": "2023MNRAS.521.4936K",
        "distmod": track.distance.distmod.value,
        "dist_ref": "2023MNRAS.521.4936K",
        "mass": 5.6e5 * u.Msun,
        "mass_ref": "2023MNRAS.521.4936K",
        "mean_width_deg": sky_width,
        "mean_width_pc": width_pc,
        "width_ref": "2023MNRAS.521.4936K",
    }
)

In [9]:
stream = mwstreams["300S-F18"]
track = stream.track.transform_to(stream.stream_frame)
track_distmod = 48.9952 - 0.2083 * stream.track.ra.degree
rows.append(
    {
        "name": "300S",
        "gs_track": "300S-F18",
        "other_names": [],
        "phi1": track.phi1.wrap_at(180 * u.deg),
        "phi2": track.phi2,
        "sky_track": coord.SkyCoord(
            phi1=track.phi1,
            phi2=track.phi2,
            distance=coord.Distance(distmod=track_distmod),
            frame=stream.stream_frame.replicate_without_data(),
        ),
        "sky_ref": "2018ApJ...866...42F",
        "distmod": track_distmod,
        "dist_ref": "2022ApJ...928...30L",
        "mass": 10**4.7 * u.Msun,
        "mass_ref": "2024MNRAS.529.2413U",
        "mean_width_deg": 0.94 * u.deg,
        "mean_width_pc": np.nan * u.pc,
        "width_ref": "2018ApJ...866...42F",
    }
)

# ---
stream = mwstreams["Willka_Yaku-S18"]
track = stream.track.transform_to(stream.stream_frame)
track_distmod = np.full(len(track), 17.8)
rows.append(
    {
        "name": "Willka Yaku",
        "gs_track": "Willka_Yaku-S18",
        "other_names": ["Willka_Yaku"],
        "phi1": track.phi1.wrap_at(180 * u.deg),
        "phi2": track.phi2,
        "sky_track": coord.SkyCoord(
            phi1=track.phi1,
            phi2=track.phi2,
            distance=coord.Distance(distmod=track_distmod),
            frame=stream.stream_frame.replicate_without_data(),
        ),
        "sky_ref": "2018ApJ...862..114S",
        "distmod": track_distmod,
        "dist_ref": "2022ApJ...928...30L",
        "mass": 14e4 * u.Msun,
        "mass_ref": "2018ApJ...862..114S",  # shipp 2018
        "mean_width_deg": np.nan * u.deg,
        "mean_width_pc": 127 * u.pc,
        "width_ref": "2018ApJ...862..114S",
    }
)

# ---
stream = mwstreams["AAU-ATLAS-L21"]
track_phi1 = np.concatenate(
    (
        mwstreams["AAU-ATLAS-L21"]
        .track.transform_to(mwstreams["AAU-ATLAS-L21"].stream_frame)
        .phi1.wrap_at(180 * u.deg)
        .degree,
        mwstreams["AAU-AliqaUma-L21"]
        .track.transform_to(mwstreams["AAU-ATLAS-L21"].stream_frame)
        .phi1.wrap_at(180 * u.deg)
        .degree,
    )
)
track_phi2 = np.concatenate(
    (
        mwstreams["AAU-ATLAS-L21"]
        .track.transform_to(mwstreams["AAU-ATLAS-L21"].stream_frame)
        .phi2.degree,
        mwstreams["AAU-AliqaUma-L21"]
        .track.transform_to(mwstreams["AAU-ATLAS-L21"].stream_frame)
        .phi2.degree,
    )
)
track_distmod = 16.67 - 0.034 * track_phi1
rows.append(
    {
        "name": "AAU",
        "gs_track": "AAU-ATLAS-L21",
        "other_names": ["ATLAS", "Aliqa Uma"],
        "phi1": track_phi1 * u.deg,
        "phi2": track_phi2 * u.deg,
        "sky_track": coord.SkyCoord(
            phi1=track_phi1 * u.deg,
            phi2=track_phi2 * u.deg,
            distance=coord.Distance(distmod=track_distmod),
            frame=mwstreams["AAU-ATLAS-L21"].stream_frame.replicate_without_data(),
        ),
        "sky_ref": "2021ApJ...911..149L",
        "distmod": track_distmod,
        "dist_ref": "2022ApJ...928...30L",
        "mass": (12e4 + 18e4) * u.Msun,  # Shipp+2018
        "mass_ref": "2018ApJ...862..114S",
        "mean_width_deg": np.nan * u.deg,
        "mean_width_pc": 96 * u.pc,
        "width_ref": "2018ApJ...862..114S",
    }
)

distmod_trends = {
    "Jet-F22": lambda x: 17.45 - 0.014 * x,
    "Phoenix-S19": lambda x: 16.26 + 0.008 * x,
    "Indus-S19": lambda x: 15.90 - 0.016 * x,
    "Palca-S18": lambda x: np.full_like(x, 17.8),
    "Elqui-S19": lambda x: 18.48 - 0.043 * x,
    "Turranburra-S19": lambda x: np.full_like(x, 17.1),
}
ref_to_bibcode = {
    "shipp2018": "2018ApJ...862..114S",
    "shipp2019": "2019ApJ...885....3S",
    "ferguson2022": "2022AJ....163...18F",
}
masses = {
    "Jet-F22": (2.5e4, "2018MNRAS.480.5342J"),
    "Phoenix-S19": (3e4, "2018ApJ...862..114S"),
    "Indus-S19": (650e4, "2018ApJ...862..114S"),
    "Palca-S18": (1e6, "2022A&A...660A..29T"),
    "Elqui-S19": (320e4, "2018ApJ...862..114S"),
    "Turranburra-S19": (180e4, "2018ApJ...862..114S"),
}
widths = {
    "Jet-F22": (
        (0.18 * u.deg * 28.6 * u.kpc).to(u.pc, u.dimensionless_angles()),
        "2018MNRAS.480.5342J",
    ),
    "Phoenix-S19": (53 * u.pc, "2018ApJ...862..114S"),
    "Indus-S19": (240 * u.pc, "2018ApJ...862..114S"),
    "Palca-S18": (300 * u.pc, "2022A&A...660A..29T"),
    "Elqui-S19": (472 * u.pc, "2018ApJ...862..114S"),
    "Turranburra-S19": (288 * u.pc, "2018ApJ...862..114S"),
}
for key, trend in distmod_trends.items():
    stream = mwstreams[key]
    name = key.split("-")[0]
    track = stream.track.transform_to(stream.stream_frame.replicate_without_data())
    track_phi1 = track.phi1.wrap_at(180 * u.deg).degree
    rows.append(
        {
            "name": name,
            "gs_track": key,
            "other_names": [] if name != "Palca" else ["Cetus"],
            "phi1": track_phi1 * u.degree,
            "phi2": track.phi2,
            "sky_track": coord.SkyCoord(
                phi1=track.phi1,
                phi2=track.phi2,
                distance=coord.Distance(distmod=trend(track_phi1)),
                frame=stream.stream_frame,
            ),
            "sky_ref": ref_to_bibcode.get(stream.ref, stream.ref),
            "distmod": trend(track_phi1),
            "dist_ref": "2022ApJ...928...30L",
            "mass": masses[key][0] * u.Msun,
            "mass_ref": masses[key][1],
            "mean_width_deg": np.nan * u.deg,
            "mean_width_pc": widths[key][0],
            "width_ref": widths[key][1],
        }
    )

s5_data = at.QTable(rows)

In [10]:
# TODO: what about the other Shipp widths? Tuc III, Molonglo, Ravi, etc.

# Orbit fitting

We fit orbits to the Ibata streams to determine distance tracks:

In [11]:
data_file = pathlib.Path("../data/stream-orbit-fits.pkl")

if not data_file.exists():
    orbitfit_data = {}

    for sid in tqdm(np.unique(ibata_tbl["sID"])):
        print(sid, sID_to_name[sid])
        mask = ibata_tbl["sID"] == sid
        orbitfit_data[sid] = run_orbit_fit(sid, ibata_g[mask], ibata_c[mask])

    with open(data_file, "wb") as f:
        pickle.dump(orbitfit_data, f)

else:
    with open(data_file, "rb") as f:
        orbitfit_data = pickle.load(f)

In [12]:
# plot_path = pathlib.Path("../plots").resolve()
# plot_path.mkdir(exist_ok=True)

# for sid, this_data in all_data.items():
#     fig, _ = make_components_plot(
#         this_data["orbit"],
#         this_data["orbit_fr"],
#         this_data["c_fr"],
#         this_data["obj_data"],
#     )
#     fig.suptitle(sID_to_name[sid], fontsize=20)
#     fig.savefig(plot_path / f"stream-{sid:04d}-orbitfit.png", dpi=200)
#     plt.close(fig)

In [13]:
ibata_fit_tbl = at.Table(
    [
        {"sID": sid, "name": sID_to_name[sid], **d["p"]}
        for sid, d in orbitfit_data.items()
    ]
)

## For cases where orbit-fitting failed, we still want to make tracks (except for Orphan, which enters in S5):

In [14]:
skip_sIDs = [
    48,  # Orphan
]

failed_sIDs = [21, 28, 77, 80, name_to_sID["New-18"]]
# fig, axes = plt.subplots(len(failed_sIDs), 1, figsize=(10, 4 * len(failed_sIDs)), layout="tight")
for i, sID in enumerate(failed_sIDs):
    if sID not in orbitfit_data:
        orbitfit_data[sID] = {}

    # ax = axes[i]
    print(sID, sID_to_name[sID])

    mask = ibata_tbl["sID"] == sID
    frame = get_frame_from_points(ibata_c[mask])
    cc = ibata_c[mask].transform_to(frame)
    _width = 1.5 * MAD(cc.phi2.degree)

    orbitfit_data[sID]["c_fr"] = cc
    orbitfit_data[sID]["p"] = None
    orbitfit_data[sID]["orbit"] = None
    orbitfit_data[sID]["orbit_fr"] = None

    # ax.scatter(cc.phi1.degree, cc.phi2.degree)
    # ax.axhspan(-_width, _width, zorder=-10, color='tab:blue')

ibata_fit_tbl = ibata_fit_tbl[~np.isin(ibata_fit_tbl["sID"], skip_sIDs)]

21 New-6


/Users/apricewhelan/projects/gala/gala/coordinates/greatcircle.py:199: RuntimeWarning: Input origin and pole are not orthogonal. Keeping the origin fixed and adjusting the pole to be orthogonal. Use warnings.simplefilter('ignore') to ignore this warning.
  warn(


28 New-9
77 NGC7099
80 New-25
56 New-18


# Internal data table: 
- Name
- sID in Ibata
- preferred galstreams track name
- sky track 
- dist/distmod track either from orbit fit or from source
- sky polygon
- stellar mass
- sources for all

Precedence is:
- S5
- Ibata atlas
- Galstreams


In [15]:
internal_data_rows = []

for tmp_row in s5_data:
    row = {}
    row["name"] = tmp_row["name"]
    row["other_names"] = tmp_row["other_names"]
    row["galstreams_track_name"] = tmp_row["gs_track"]

    row["track"] = tmp_row["sky_track"]
    row["sky_track_ref"] = tmp_row["sky_ref"]
    row["dist_track_ref"] = tmp_row["dist_ref"]

    row["stellar_mass"] = tmp_row["mass"]
    row["stellar_mass_ref"] = tmp_row["mass_ref"]

    mean_dist = np.mean(row["track"].distance)
    if np.isnan(tmp_row["mean_width_deg"]):
        row["width_sky"] = (tmp_row["mean_width_pc"] / mean_dist).to(
            u.deg, u.dimensionless_angles()
        )
    else:
        row["width_sky"] = tmp_row["mean_width_deg"]

    if np.isnan(tmp_row["mean_width_pc"]):
        row["width"] = (row["width_sky"] * mean_dist).to(u.pc, u.dimensionless_angles())
    else:
        row["width"] = tmp_row["mean_width_pc"]

    row["width_ref"] = tmp_row["width_ref"]

    # Sky polygon:
    if row["name"] == "AAU":
        # Special case: get sky track for AAU from Ibata member stars
        d = orbitfit_data[name_to_sID["ATLAS"]]
        _, row["sky_poly"] = make_ibata_poly_nodes(d["c_fr"])

    else:
        row["sky_poly"] = get_full_galstreams_poly(
            mwstreams[tmp_row["gs_track"]].poly_sc
        )

    internal_data_rows.append(row)

internal_data = at.QTable(internal_data_rows)

Now match S5 streams to Ibata atlas stream IDs:

In [16]:
ibata_names = np.asarray(np.unique(ibata_tbl["name"]))
ibata_sIDs = []
for row in internal_data:
    row_names = [row["name"]] + row["other_names"]
    for name in row_names:
        if name in ibata_names:
            sID = name_to_sID[name]
            break
    else:
        sID = 0
    ibata_sIDs.append(sID)
internal_data["ibata2023_sID"] = ibata_sIDs

In [17]:
ibata_to_galstreams = {
    "Tuc3": "TucanaIII",
    "NGC7099": "M30",
    "M5": "NGC5904",
    "M92": "NGC6341",
}

In [18]:
for name in ibata_names:
    sID = name_to_sID[name]
    if sID in internal_data["ibata2023_sID"]:
        print(sID, name)
        continue

    row = {}
    row["name"] = name
    row["other_names"] = (
        [] if name not in ibata_to_galstreams else [ibata_to_galstreams[name]]
    )

    track_name, _ = get_default_track_for_stream(name)
    if track_name is None:
        track_name, _ = get_default_track_for_stream(
            ibata_to_galstreams.get(name, name)
        )
    row["galstreams_track_name"] = track_name if track_name is not None else ""
    row["ibata2023_sID"] = sID

    if sID in skip_sIDs:
        continue

    if sID not in orbitfit_data:
        print("NOT IN ORBITFIT", sID, row["name"])
        continue

    this_data = orbitfit_data[sID]
    sky_track, row["sky_poly"] = make_ibata_poly_nodes(this_data["c_fr"])

    row["sky_track_ref"] = "2023arXiv231117202I"
    row["dist_track_ref"] = ""  # TODO: this paper
    if this_data["orbit_fr"] is not None:
        x = this_data["orbit_fr"].phi1.degree
        spl = InterpolatedUnivariateSpline(
            x[np.argsort(x)], this_data["orbit_fr"].distance.kpc[np.argsort(x)], k=3
        )

        row["track"] = coord.SkyCoord(
            phi1=sky_track.phi1,
            phi2=sky_track.phi2,
            distance=spl(sky_track.phi1.degree) * u.kpc,
            frame=sky_track.frame.replicate_without_data(),
        )
    else:
        if "obj_data" in this_data:
            _d = this_data["obj_data"]
            mean_plx = np.sum(_d["parallax"] / _d["parallax_error"] ** 2) / np.sum(
                1 / _d["parallax_error"] ** 2
            )
            dist = 1 / mean_plx * u.kpc
        else:
            dist = np.median(this_data["c_fr"].distance)

        row["track"] = coord.SkyCoord(
            phi1=sky_track.phi1,
            phi2=sky_track.phi2,
            distance=dist,
            frame=sky_track.frame.replicate_without_data(),
        )
        row["dist_track_ref"] = "2023arXiv231117202I"

    # Estimate stellar mass using an isochrone:
    DM = np.mean(row["track"].distance.distmod.value)
    MSTO_r = DM + MSTO_absmag

    idx = np.where((iso["G"] + DM) < 20.0)[0]
    dIMF = iso["int_IMF"][idx[-1]] - iso["int_IMF"][idx[0]]
    M = 2.0 * len(d["c_fr"]) / dIMF

    row["stellar_mass"] = M * u.Msun
    row["stellar_mass_ref"] = ""  # TODO: this paper

    # Estimate stream width:
    xx = row["track"].phi1.wrap_at(180 * u.deg).degree
    _spl = InterpolatedUnivariateSpline(
        xx[np.argsort(xx)], row["track"].phi2.degree[np.argsort(xx)], k=3
    )
    c_track_phi2 = _spl(this_data["c_fr"].phi1.wrap_at(180 * u.deg).degree)
    row["width_sky"] = 1.5 * MAD(this_data["c_fr"].phi2.degree - c_track_phi2) * u.deg
    row["width"] = (row["width_sky"] * np.mean(row["track"].distance)).to(
        u.pc, u.dimensionless_angles()
    )
    row["width_ref"] = ""  # TODO: this work

    internal_data.add_row(row)

4 ATLAS
Initializing galstreams library from master_log... 
15 Indus
48 Orphan
5 Phoenix


Now find all other streams in Galstreams that are not already in our internal table:

In [19]:
galstreams_masses = {
    "20.0-1": (1e5 * u.Msun, "mateu2018"),  # Estimate based on MV=-7.3,
    "Corvus": (1e5 * u.Msun, "mateu2018"),  # Estimate based on MV=-7.5,
    "PS1-A": (0.22e3 * 2 * u.Msun, "bernard2016"),  # M/L = 2
    "PS1-B": (1.2e3 * 2 * u.Msun, "bernard2016"),
    "PS1-C": (1e3 * 2 * u.Msun, "bernard2016"),
    "PS1-D": (7.5e3 * 2 * u.Msun, "bernard2016"),
    "PS1-E": (0.5e3 * 2 * u.Msun, "bernard2016"),
    "Pal13": (1.4e3 * 2 * u.Msun, "shipp2020"),  # M/L = 2, Lv in tails from RRL
    "Pegasus": (2e3 * 2 * u.Msun, "perottoni2019"),  # M/L = 2
    "Ravi": (1e4 * u.Msun, "shipp2018"),
    "Turbio": (3.5e3 * u.Msun, "shipp2018"),
    "Wambelong": (1.6e3 * u.Msun, "shipp2018"),
}

# No reported masses:
# Grillmair 2009: Acheron, Cocytos, Lethe, Styx
# Grillmair 2013: Alpheus
# Williams+2011: Aquarius
# Ibata+2021: C-4, C-5, C-8, Gaia-2, Gaia-3, Gaia-4, Gaia-5, Gunnthra, M2
# Myeong+2017: Eridanus, Pal15
# Grillmair 2014: Hermus, Hyllus
# Sollima 2020: M30, NGC6362
# Grillmair 2017a: Sangarius, Scamander
# Grillmair 2017b: Murrumbidgee, Molonglo, Orinoco, Kwando
# Bonaca+2012: Tri-Pis
# Weiss+2018: Parallel, Perpendicular

In [20]:
galstreams_widths = {
    "20.0-1": (1.8 * u.deg, 641 * u.pc, "mateu2018"),
    "Corvus": (1.63 * u.deg, 313 * u.pc, "mateu2018"),
    "PS1-A": (27 * u.arcmin, 63 * u.pc, "bernard2016"),
    "PS1-B": (27 * u.arcmin, 112 * u.pc, "bernard2016"),
    "PS1-C": (20 * u.arcmin, 99 * u.pc, "bernard2016"),
    "PS1-D": (52 * u.arcmin, 350 * u.pc, "bernard2016"),
    "PS1-E": (37 * u.arcmin, 140 * u.pc, "bernard2016"),
    "Pal13": (0.25 * u.deg, (0.25 * u.deg * 23.6 * u.kpc), "shipp2020"),
    "Pegasus": (0.4 * u.deg, 112 * u.pc, "perottoni2019"),
    "Ravi": (288 * u.pc / (22.9 * u.kpc), 288 * u.pc, "shipp2018"),
    "Turbio": (72 * u.pc / (16.6 * u.kpc), 72 * u.pc, "shipp2018"),
    "Wambelong": (106 * u.pc / (15.1 * u.kpc), 106 * u.pc, "shipp2018"),
    "Acheron": (0.9 * u.deg, 60 * u.pc, "grillmair2009"),
    "Cocytos": (0.7 * u.deg, 140 * u.pc, "grillmair2009"),
    "Lethe": (0.4 * u.deg, 95 * u.pc, "grillmair2009"),
    "Styx": (3.3 * u.deg, 2.6 * u.kpc, "grillmair2009"),
    "Alpheus": (3.2 * u.deg, 100 * u.pc, "grillmair2013"),
    "Hermus": (0.7 * u.deg, 220 * u.pc, "grillmair2014"),
    "Hyllus": (1.2 * u.deg, 180 * u.pc, "grillmair2014"),
    "Sangarius": (1 * u.deg, 350 * u.pc, "grillmair2017a"),
    "Scamander": (1 * u.deg, 350 * u.pc, "grillmair2017a"),
    "Molonglo": (30 * u.arcmin, 170 * u.pc, "grillmair2017b"),
    "Murrumbidgee": (22 * u.arcmin, 125 * u.pc, "grillmair2017b"),
    "Orinoco": (40 * u.arcmin, 240 * u.pc, "grillmair2017b"),
    "Kwando": (22 * u.arcmin, 130 * u.pc, "grillmair2017b"),
    "Tri-Pis": (0.2 * u.deg, 75 * u.pc, "bonaca2012"),
}

# No reported widths:
# Williams+2011: Aquarius
# Myeong+2017: Eridanus, Pal15
# Sollima 2020: M30, NGC6362
# Weiss+2018: Parallel, Perpendicular

In [21]:
gs_to_internal_name = {
    "AAU-ATLAS": "AAU",
    "AAU-AliqaUma": "AAU",
    "M68-Fjorm": "Fjorm",
    "NGC3201-Gjoll": "Gjoll",
    "OmegaCen-Fimbulthul": "Fimbulthul",
    "Orphan-Chenab": "OC",
    "TucanaIII": "Tuc3",
    "Jhelum-a": "Jhelum",
    "Jhelum-b": "Jhelum",
}

galstreams_default_tracks = np.array(list(mwstreams.keys()))
galstreams_names = np.array(
    [
        x
        for x in mwstreams.all_unique_stream_names()
        if x not in ["Cetus-New", "Cetus-Palca", "ACS", "Monoceros", "Sagittarius"]
    ]
)

_all_internal_names = np.concatenate(
    (internal_data["name"], [x for y in internal_data["other_names"] for x in y])
)
for name in galstreams_names:
    if gs_to_internal_name.get(name, name) in _all_internal_names:
        continue

    row = {}
    row["name"] = name
    row["other_names"] = []

    track_name, stream = get_default_track_for_stream(name)
    row["galstreams_track_name"] = track_name if track_name is not None else ""

    row["sky_poly"] = get_full_galstreams_poly(stream.poly_sc)
    row["track"] = stream.track.transform_to(
        stream.stream_frame.replicate_without_data()
    )
    row["sky_track_ref"] = stream.ref
    row["dist_track_ref"] = stream.ref

    if name in galstreams_masses:
        row["stellar_mass"] = galstreams_masses[name][0]
        row["stellar_mass_ref"] = galstreams_masses[name][1]
    else:
        row["stellar_mass"] = np.nan * u.Msun
        row["stellar_mass_ref"] = ""

    if name in galstreams_widths:
        _w = galstreams_widths[name]
        row["width_sky"] = _w[0].to(u.deg, u.dimensionless_angles())
        row["width"] = _w[1].to(u.pc, u.dimensionless_angles())
        row["width_ref"] = _w[2]
    else:
        print(name)
        row["width_sky"] = np.nan * u.deg
        row["width"] = np.nan * u.pc
        row["width_ref"] = ""

    internal_data.add_row(row)

Aquarius
C-4
C-5
C-8
Eridanus
Gaia-2
Gaia-3
Gaia-4
Gaia-5
Gunnthra
M2
NGC6362
Pal15
Parallel
Perpendicular


Final adjustments to fill in values in columns:

In [22]:
paper_to_bibcode = {
    "bernard2016": "2016MNRAS.463.1759B",
    "bonaca2012": "2012ApJ...760L...6B",
    "grillmair2009": "2009ApJ...693.1118G",
    "grillmair2013": "2013ApJ...769L..23G",
    "grillmair2014": "2014ApJ...790L..10G",
    "grillmair2017": "2017ApJ...834...98G",
    "grillmair2017a": "2017ApJ...834...98G",
    "grillmair2017b": "2017ApJ...847..119G",
    "ibata2021": "2021ApJ...914..123I",
    "malhan2018": "2018MNRAS.481.3442M",
    "mateu2018": "2018MNRAS.474.4112M",
    "myeong2017": "2017MNRAS.469L..78M",
    "perottoni2019": "2019MNRAS.486..843P",
    "shipp2018": "2018ApJ...862..114S",
    "shipp2020": "2020AJ....160..244S",
    "sollima2020": "2020MNRAS.495.2222S",
    "weiss2018": "2018ApJ...867L...1W",
    "williams2011": "2011ApJ...728..102W",
    "bonaca2012": "2012ApJ...760L...6B",
}
for colname in ["sky_track_ref", "dist_track_ref", "stellar_mass_ref", "width_ref"]:
    internal_data[colname] = [
        paper_to_bibcode.get(x, x) for x in internal_data[colname]
    ]

## Discovery methods

In [23]:
discovery_method = {
    "Acheron": "photometry",
    "Styx": "photometry",
    "Cocytos": "photometry",
    "Lethe": "photometry",
    "Aquarius": "photometry",
    "Tri-Pis": "photometry",
    "Alpheus": "photometry",
    "Hermus": "photometry",
    "Hyllus": "photometry",
    "PS1-C": "photometry",
    "PS1-D": "photometry",
    "PS1-E": "photometry",
    "PS1-A": "photometry",
    "PS1-B": "photometry",
    "Orinoco": "photometry",
    "Sangarius": "photometry",
    "Scamander": "photometry",
    "Molonglo": "photometry",
    "Murrumbidgee": "photometry",
    "Pal15": "photometry",
    "Eridanus": "photometry",
    "Wambelong": "photometry",
    "Ravi": "photometry",
    "Palca": "photometry",
    "Willka Yaku": "photometry",
    "Turbio": "photometry",
    "300S": "photometry",
    "Parallel": "photometry",
    "Perpendicular": "photometry",
    "Corvus": "rrl",
    "20.0-1": "rrl",
    "Gaia-5": "streamfinder",
    "Gaia-4": "streamfinder",
    "Gaia-3": "streamfinder",
    "Turranburra": "photometry",
    "Elqui": "photometry",
    "Indus": "photometry",
    "Phoenix": "photometry",
    "OC": "photometry",
    "Pegasus": "photometry",
    "Pal13": "photometry",
    "M30": "photometry",
    "NGC6362": "astrometry",
    "AAU": "photometry",
    "M2": "streamfinder",
    "Gaia-2": "streamfinder",
    "C-8": "streamfinder",
    "C-5": "streamfinder",
    "C-4": "streamfinder",
    "Gunnthra": "streamfinder",
    "Jet": "photometry",
    "Gjoll": "streamfinder",
    "Gaia-10": "streamfinder",
    "Hrid": "streamfinder",
    "Hydrus": "photometry",
    "Gaia-8": "streamfinder",
    "Gaia-7": "streamfinder",
    "Gaia-6": "streamfinder",
    "Ylgr": "streamfinder",
    "Tuc3": "photometry",
    "Gaia-12": "streamfinder",
    "Gaia-11": "streamfinder",
    "Gaia-9": "streamfinder",
    "Kwando": "photometry",
    "GD-1": "photometry",
    "C-10": "streamfinder",
    "C-11": "streamfinder",
    "C-12": "streamfinder",
    "C-13": "streamfinder",
    "C-19": "streamfinder",
    "C-20": "streamfinder",
    "C-22": "streamfinder",
    "Gaia-1": "streamfinder",
    "C-23": "streamfinder",
    "C-25": "streamfinder",
    "C-7": "streamfinder",
    "C-9": "streamfinder",
    "Fimbulthul": "streamfinder",
    "Fimbulthul-S": "streamfinder",
    "Sylgr": "streamfinder",
    "Fjorm": "streamfinder",
    "C-24": "streamfinder",
    "Svol": "streamfinder",
    "Slidr": "streamfinder",
    "Phlegethon": "streamfinder",
    "New-11": "streamfinder",
    "New-10": "streamfinder",
    "New-1": "streamfinder",
    "NGC7492": "streamfinder",
    "NGC7089": "streamfinder",
    "NGC6397": "streamfinder",
    "NGC6101": "streamfinder",
    "NGC5466": "photometry",
    "NGC288": "astrometry",
    "NGC2808": "streamfinder",
    "NGC2298": "astrometry",
    "NGC1851": "streamfinder",
    "NGC1261b": "streamfinder",
    "NGC1261a": "streamfinder",
    "NGC1261": "streamfinder",
    "M92": "astrometry",
    "M5": "photometry",
    "Leiptr": "streamfinder",
    "LMS-1": "iom-clustering",
    "New-12": "streamfinder",
    "Kshir": "streamfinder",
    "New-13": "streamfinder",
    "New-15": "streamfinder",
    "Pal5": "photometry",
    "Ophiuchus": "photometry",
    "New-8": "streamfinder",
    "New-7": "streamfinder",
    "New-6": "streamfinder",
    "New-5": "streamfinder",
    "New-4": "streamfinder",
    "New-3": "streamfinder",
    "New-28": "streamfinder",
    "New-27": "streamfinder",
    "New-26": "streamfinder",
    "New-24": "streamfinder",
    "New-23": "streamfinder",
    "New-22": "streamfinder",
    "New-21": "streamfinder",
    "New-20": "streamfinder",
    "New-2": "streamfinder",
    "New-19": "streamfinder",
    "New-18": "streamfinder",
    "Jhelum": "photometry",
    "New-16": "streamfinder",
    "New-14": "streamfinder",
    "New-17": "streamfinder",
}
discovery_method = at.Table(
    {
        "name": list(discovery_method.keys()),
        "discovery_method": list(discovery_method.values()),
    }
)

In [24]:
internal_data = at.join(internal_data, discovery_method, keys="name", join_type="left")

## Spectroscopic data:

In [25]:
paper_to_bibcode = {
    "bonaca2012": "2012ApJ...760L...6B",
    "bonaca2020": "2020ApJ...892L..37B",
    "caldwell2020": "2020AJ....159..287C",
    "ferguson2022": "2022AJ....163...18F",
    "fu2018": "2018ApJ...866...42F",
    "gialluca2021": "2021ApJ...911L..32G",
    "hansen2021": "2021ApJ...915..103H",
    "ibata2017": "2017ApJ...842..120I",
    "ibata2019": "2019ApJ...872..152I",
    "ibata2019b": "2019NatAs...3..667I",
    "ibata2021": "2021ApJ...914..123I",
    "ibata2023": "",
    "jensen2021": "2021MNRAS.507.1923J",
    "ji2020": "2020AJ....160..181J",
    "koposov2010": "2010ApJ...712..260K",
    "koposov2019": "2019MNRAS.485.4726K",
    "koposov2023": "2023MNRAS.521.4936K",
    "kuzma2015": "2015MNRAS.446.3297K",
    "li2019": "2019MNRAS.490.3508L",
    "li2021": "2021ApJ...911..149L",
    "li2022": "2022ApJ...928...30L",
    "malhan2019": "2019ApJ...886L...7M",
    "martin2013": "2013ApJ...765L..39M",
    "odenkirchen2009": "2009AJ....137.3378O",
    "pricewhelan2019": "2019AJ....158..223P",
    "sesar2015": "2015ApJ...809...59S",
    "sheffield2021": "2021ApJ...913...39S",
    "shipp2019": "2019ApJ...885....3S",
    "williams2011": "2011ApJ...728..102W",
    "yuan2020": "2020ApJ...898L..37Y",
}

spec_data = {
    "300S": ["fu2018", "li2022"],
    "AAU": ["li2019", "ji2020", "li2022", "ibata2023"],
    "GD-1": ["koposov2010", "bonaca2020", "gialluca2021"],
    "Elqui": ["li2019", "ji2020", "li2022"],
    "Fimbulthul": ["ibata2019", "ibata2019b"],
    "Fjorm": ["ibata2019"],
    "Gjoll": ["ibata2019", "ibata2021"],
    "Gunnthra": ["ibata2021"],
    "Hrid": ["ibata2021"],
    "Indus": ["li2019", "ji2020", "hansen2021", "li2022"],
    "Jet": ["li2022"],
    "Jhelum": ["li2019", "ji2020", "sheffield2021", "li2022"],
    "Kshir": ["malhan2019"],
    "Leiptr": ["ibata2019"],
    "LMS-1": ["yuan2020"],
    "NGC3201": ["ibata2021"],
    "NGC6397": ["ibata2021"],
    "OC": ["li2019", "ji2020", "li2022", "koposov2023", "ibata2023"],
    "Ophiuchus": ["sesar2015", "caldwell2020", "li2022"],
    "Pal5": ["odenkirchen2009", "kuzma2015", "ibata2017"],
    "Palca": ["li2022"],
    "Phoenix": ["li2019", "ji2020", "li2022"],
    "Slidr": ["ibata2019"],
    "Slygr": ["ibata2019"],
    "Tri-Pis": ["martin2013"],
    "Turranburra": ["li2022"],
    "Willka Yaku": ["li2022"],
    "Ylgr": ["ibata2019"],
}

# Add in all streams observed in Ibata+2023 based on source IDs from paper
for name in np.unique(
    ibata_tbl["name"][np.isin(ibata_tbl["vsource"], [8, 12, 13, 14])]
):
    if name not in internal_data["name"]:
        continue

    if name not in spec_data:
        spec_data[name] = []

    spec_data[name].append("ibata2023")

spec_data = at.Table(
    [
        {
            "name": k,
            "has_spec": True,
            "spec_bibcodes": [paper_to_bibcode[y] for y in v],
        }
        for k, v in spec_data.items()
    ]
)

internal_data = at.join(internal_data, spec_data, keys="name", join_type="left")
internal_data["has_spec"] = internal_data["has_spec"].filled(False)
internal_data["spec_bibcodes"] = internal_data["spec_bibcodes"].filled("")

In [26]:
internal_data[~internal_data["has_spec"]]["name"].pprint(1000)

     name    
-------------
       20.0-1
      Acheron
      Alpheus
     Aquarius
         C-13
         C-22
         C-23
         C-24
          C-4
          C-5
          C-8
      Cocytos
       Corvus
     Eridanus
      Gaia-10
       Gaia-2
       Gaia-3
       Gaia-4
       Gaia-5
       Gaia-6
       Hermus
       Hydrus
       Hyllus
        Lethe
           M2
     Molonglo
 Murrumbidgee
      NGC1261
     NGC1261a
     NGC1261b
      NGC1851
      NGC2298
      NGC5466
      NGC6362
      NGC7492
        New-1
       New-10
       New-12
       New-18
       New-19
       New-20
        New-9
      Orinoco
        PS1-A
        PS1-B
        PS1-C
        PS1-D
        PS1-E
        Pal13
        Pal15
     Parallel
      Pegasus
Perpendicular
         Ravi
    Sangarius
    Scamander
         Styx
         Svol
       Turbio
    Wambelong


## Detected in Gaia:

In [27]:
gaia_override = [
    "300S",
    "AAU",
    "Elqui",
    "Gaia-3",
    "Gaia-4",
    "Gaia-5",
    "Jet",
    "OC",
    "Palca",
    "Turranburra",
    "Willka Yaku",
]

has_gaia = []
for irow in internal_data:
    try:
        pm1 = irow["track"].pm_phi1_cosphi2
        pm_track_bool = np.any(~np.isclose(pm1.value, 0))
    except TypeError:
        pm_track_bool = False

    has_gaia.append(
        (
            (irow["name"] in ibata_tbl["name"])
            or pm_track_bool
            or (irow["name"] in gaia_override)
        )
        and irow["name"] != "Aquarius"
    )

internal_data["has_gaia"] = has_gaia

In [28]:
internal_data[internal_data["has_gaia"]]["name"].pprint(1000)

    name    
------------
        300S
         AAU
        C-10
        C-11
        C-12
        C-13
        C-19
        C-20
        C-22
        C-23
        C-24
        C-25
         C-4
         C-5
         C-7
         C-8
         C-9
       Elqui
  Fimbulthul
Fimbulthul-S
       Fjorm
        GD-1
      Gaia-1
     Gaia-10
     Gaia-11
     Gaia-12
      Gaia-2
      Gaia-3
      Gaia-4
      Gaia-5
      Gaia-6
      Gaia-7
      Gaia-8
      Gaia-9
       Gjoll
    Gunnthra
        Hrid
      Hydrus
       Indus
         Jet
      Jhelum
       Kshir
      Kwando
       LMS-1
      Leiptr
          M2
          M5
         M92
     NGC1261
    NGC1261a
    NGC1261b
     NGC1851
     NGC2298
     NGC2808
      NGC288
     NGC5466
     NGC6101
     NGC6397
     NGC7089
     NGC7099
     NGC7492
       New-1
      New-10
      New-11
      New-12
      New-13
      New-14
      New-15
      New-16
      New-17
      New-18
      New-19
       New-2
      New-20
      New-21

In [29]:
internal_data[~internal_data["has_gaia"]]["name"].pprint(1000)

     name    
-------------
       20.0-1
      Acheron
      Alpheus
     Aquarius
      Cocytos
       Corvus
     Eridanus
       Hermus
       Hyllus
        Lethe
     Molonglo
 Murrumbidgee
      NGC6362
      Orinoco
        PS1-A
        PS1-B
        PS1-C
        PS1-D
        PS1-E
        Pal13
        Pal15
     Parallel
      Pegasus
Perpendicular
    Sangarius
    Scamander
         Styx
      Tri-Pis


In [30]:
with open("../data/all_stream_summary_data.pkl", "wb") as f:
    pickle.dump(internal_data, f)

# Main Summary Table

- name
- phi1,2 = (0, 0) in ICRS coords
- length
- distmod (dist)
- range of distmod (dist)
- stellar mass
- found with gaia y/n
- spec follow up y/n
- associated with progenitor y/n

In [90]:
summary_table = []
for irow in internal_data:
    row = {}
    row["Name"] = irow["name"]

    frame = irow["track"].frame.replicate_without_data()

    _sort_idx = np.argsort(irow["track"].phi1.degree)
    dist_spl = InterpolatedUnivariateSpline(
        irow["track"].phi1.degree[_sort_idx], irow["track"].distance.kpc[_sort_idx], k=3
    )
    origin_c = coord.SkyCoord(
        phi1=0 * u.deg,
        phi2=0 * u.deg,
        distance=np.abs(dist_spl(0.0)) * u.kpc,
        frame=frame,
    )
    origin_c = origin_c.transform_to(coord.ICRS())
    galcen = origin_c.transform_to(coord.Galactocentric())
    row["Origin RA"] = origin_c.ra
    row["Origin Dec."] = origin_c.dec

    # dhelio_name = "Origin Helio. Dist."
    dhelio_name = r"$D_{\rm hel}$"
    row[dhelio_name] = origin_c.distance.view(u.Quantity)

    # rgal_name = "Origin Gal. Radius"
    rgal_name = r"$r_{\rm gal}$"
    row[rgal_name] = np.squeeze(np.linalg.norm(galcen.data.xyz))

    # if np.isclose(irow["track"].distance.kpc.min(), irow["track"].distance.kpc.max()):
    #     row["Min. Dist."] = np.nan * u.kpc
    #     row["Max. Dist."] = np.nan * u.kpc
    # else:
    #     row["Min. Dist."] = irow["track"].distance.min().view(u.Quantity)
    #     row["Max. Dist."] = irow["track"].distance.max().view(u.Quantity)

    row["Length (sky)"] = irow["track"].phi1.max() - irow["track"].phi1.min()
    row["Width (sky)"] = irow["width_sky"]
    row["Width"] = irow["width"]
    row["Reference"] = irow["sky_track_ref"]
    
    row["Width Ref."] = irow["width_ref"] if irow["width_ref"] != "" else "this work"
    if np.isnan(irow["width"]):
        row["Width Ref."] = ""
                
    row["Stellar Mass"] = irow["stellar_mass"]
    if irow["stellar_mass_ref"] != "" or np.isnan(irow["stellar_mass"]):
        row["Stellar Mass Ref."] = irow["stellar_mass_ref"]
    else:
        row["Stellar Mass Ref."] = "this work"

    if irow["ibata2023_sID"] != 0:
        row["Distance Ref."] = "this work"
    else:
        row["Distance Ref."] = irow["sky_track_ref"]

    row["Found in Gaia"] = irow["has_gaia"]
    row["Spec. Follow Up"] = irow["has_spec"]
    row["Spec. Ref."] = irow["spec_bibcodes"]
    # row["Gaia"] = irow["ibata2023_sID"] != 0 or pm_track_bool
    # row["S5"] = np.in1d([irow["name"]] + irow["other_names"], s5_data["name"]).any()
    summary_table.append(row)

tmp_summary_table = at.QTable(summary_table)

Sort by: (spec and gaia, gaia, all remaining), and then alphabetical by name within each:

In [91]:
# Custom sort function:
def natural_sort(key):
    return [int(s) if s.isdigit() else s for s in re.split(r"(\d+)", key)]


tbl1 = tmp_summary_table[tmp_summary_table["Spec. Follow Up"] & tmp_summary_table["Found in Gaia"]]
tbl2 = tmp_summary_table[~tmp_summary_table["Spec. Follow Up"] & tmp_summary_table["Found in Gaia"]]
tbl3 = tmp_summary_table[~np.isin(tmp_summary_table["Name"], tbl1["Name"]) & ~np.isin(tmp_summary_table["Name"], tbl2["Name"])]
assert len(tbl1) + len(tbl2) + len(tbl3) == len(tmp_summary_table)
tbls = [tbl1, tbl2, tbl3]
table_heads = [
    "Has spectroscopic follow up and Gaia detection",
    "Gaia detection",
    "Needs follow-up observations",
]

sorted_tbls = []
split_streams = []
for tbl in tbls:
    sorted_names = sorted(tbl["Name"], key=natural_sort)
    argsort = np.array([np.where(tbl["Name"] == name)[0][0] for name in sorted_names])
    tbl = tbl[argsort]
    sorted_tbls.append(tbl)
    split_streams.append(tbl["Name"][-1])

summary_table = at.vstack(sorted_tbls)
summary_table.remove_columns(["Found in Gaia", "Spec. Follow Up"])

In [68]:
bibcode_to_bib = {}
bibcode_to_ref_id = {}
i = 1
for bibcode in np.unique(
    np.concatenate(
        (
            summary_table["Reference"],
            summary_table["Stellar Mass Ref."],
            summary_table["Distance Ref."],
            summary_table["Width Ref."],
            [x for y in summary_table["Spec. Ref."][summary_table["Spec. Ref."] != ""] for x in y]
        )
    )
):
    if bibcode == "":
        continue

    try:
        paper = list(ads.SearchQuery(bibcode=bibcode))[0]
    except IndexError:
        print(bibcode)
        continue

    # Handle grillmair 2017b
    cite_key = f"{paper.first_author.split(',')[0].lower()}:{paper.year}"
    if cite_key in [x[0] for x in bibcode_to_bib.values()]:
        cite_key = f"{cite_key}b"
    bibtex = paper.bibtex.replace(bibcode, cite_key)
    bibcode_to_bib[bibcode] = (cite_key, bibtex)
    bibcode_to_ref_id[bibcode] = i
    i += 1

bibcode_to_ref_id["this work"] = "*"

/Users/apricewhelan/anaconda3/lib/python3.10/site-packages/ads/utils.py:26: UserWarning: You are lazy loading attributes via 'bibtex', and so are making multiple calls to the API. This will impact your overall rate limits.
  warnings.warn(
/Users/apricewhelan/anaconda3/lib/python3.10/site-packages/ads/search.py:302: UserWarning: bibtex should be queried with ads.ExportQuery(); You will hit API ratelimits very quickly otherwise.
  warnings.warn("bibtex should be queried with ads.ExportQuery(); You will "


this work


Replace references with letters after the stream name:

In [92]:
# Add reference letters next to values:
# with_refs = {"Name": [], dhelio_name: [], "Stellar Mass": []}
# for row in summary_table:
#     ref = bibcode_to_ref_id.get(row["Reference"], "")
#     with_refs["Name"].append(f"{row['Name']}$^{{({ref})}}$" if ref else "")

#     d = row[dhelio_name]
#     ref = bibcode_to_ref_id.get(row["Distance Ref."], "")
#     with_refs[dhelio_name].append(
#         f"{d.value:.1f}$^{{({ref})}}$" if ref else f"{d.value:.1f}"
#     )

#     ref = bibcode_to_ref_id.get(row["Stellar Mass Ref."], "")
#     with_refs["Stellar Mass"].append(
#         f"{row['Stellar Mass'].value:.0e}$^{{({ref})}}$"
#         if ref
#         else f"{row['Stellar Mass'].value:.0e}"
#     )

# summary_table["Name"] = with_refs["Name"]
# summary_table[dhelio_name] = with_refs[dhelio_name]
# summary_table[dhelio_name].unit = u.kpc
# summary_table["Stellar Mass"] = with_refs["Stellar Mass"]
# summary_table["Stellar Mass"].unit = u.Msun
# summary_table.remove_columns(["Reference", "Stellar Mass Ref.", "Distance Ref."])

# Add a new column with all reference letters:
refs = []
for row in summary_table:
    ref = bibcode_to_ref_id.get(row["Reference"], "")
    ref_d = bibcode_to_ref_id.get(row["Distance Ref."], "")
    ref_m = bibcode_to_ref_id.get(row["Stellar Mass Ref."], "")
    ref_w = bibcode_to_ref_id.get(row["Width Ref."], "")
    ref_s = [bibcode_to_ref_id.get(x, "") for x in row["Spec. Ref."]] if row["Spec. Ref."] else []
    refs.append(
        ",".join([f"({r})" for r in np.unique([ref, ref_w, ref_d, ref_m] + ref_s) if r])
    )

final_summary_table = summary_table.copy()
final_summary_table["References"] = refs
final_summary_table.remove_columns(
    ["Reference", "Stellar Mass Ref.", "Distance Ref.", "Width Ref.", "Spec. Ref."]
)

In [93]:
caption = [
    f"({num}): \citet{{{bibcode_to_bib[bibcode][0]}}}"
    if bibcode in bibcode_to_bib
    else f"({num}): this work"
    for bibcode, num in bibcode_to_ref_id.items()
]
caption = ", ".join(caption)

Split the full table into batches:

In [94]:
idx = np.arange(0, len(final_summary_table) + 1, 34)
if idx[-1] < len(final_summary_table):
    idx = np.concatenate((idx, [len(final_summary_table)]))

sub_tables = [final_summary_table[i:j] for i, j in zip(idx[:-1], idx[1:])]
[len(x) for x in sub_tables]

[34, 34, 34, 31]

In [95]:
texts = []
for i, tbl in enumerate(sub_tables):
    with io.StringIO() as f:
        tbl.write(
            f,
            format="ascii.latex",
            overwrite=True,
            formats={
                "Origin RA": "%.3f",
                "Origin Dec.": "%.3f",
                dhelio_name: "%.1f",
                rgal_name: "%.1f",
                "Length (sky)": "%.0f",
                "Width (sky)": "%.1f",
                "Width": "%.0f",
                "Stellar Mass": "%.0e",
            },
            latexdict={
                "header_start": r"\hline \hline",
                "data_start": r"\hline",
                "data_end": r"\hline \hline",
            },
            col_align="cSSSScSccccc",
        )

        text = f.getvalue()
        text = text.replace(" nan ", " ")

        text = re.sub("(\d+)e\+0(\d+)", r"$\1 \\times 10^{\2}$", text)
        # text = text.replace("_", "")
        texts.append(text)

In [96]:
unit_to_siunitx = {
    u.deg: "\\unit{\\degree}",
    u.kpc: "\\unit{\\kilo\\parsec}",
    u.pc: "\\unit{\\parsec}",
    u.Msun: "\\unit{\\Msun}",
}

In [97]:
for i, text in enumerate(texts, start=1):
    for bibcode, (cite_key, bibtex) in bibcode_to_bib.items():
        text = text.replace(bibcode, f"\\citet{{{cite_key}}}")

    split_text = text.split("\n")

    # Fix the column names with periods for aligning on decimal:
    # TODO: change replace to rstrip
    new_header = " & ".join(
        [f"{{{x.strip()}}}" for x in split_text[3].rstrip("\\").split(" & ")]
    )
    split_text[3] = new_header + "\\\\"

    # Fix the units to use siunitx:
    new_units = " & ".join(
        [
            unit_to_siunitx.get(col.unit, "")
            for col in final_summary_table.columns.values()
        ]
    )
    split_text[4] = new_units + "\\\\"

    # Table head for first block:
    if i == 1:
        split_text[5] = split_text[5] + "\\\\"
        head = table_heads[0]
        split_text.insert(
            6,
            f"\\multicolumn{{{len(final_summary_table.colnames)}}}{{l}}"
            f"{{\\bf {head}:}}\\\\[1pt]",
        )

    # Add hlines below any lines that start with the "split stream" names from above:
    lines = []
    for line in split_text:
        lines.append(line)
        for name, head in zip(split_streams[:-1], table_heads[1:]):
            if line.strip().startswith(name):
                lines.append("\\hline \\\\")
                lines.append(
                    f"\\multicolumn{{{len(final_summary_table.colnames)}}}{{l}}"
                    f"{{\\bf {head}:}}\\\\[1pt]"
                )

    text = "\n".join(lines)

    with open(f"../tex/figures/stream-summary-table-{i}.tex", "w") as f:
        f.write(text)

In [98]:
with open(f"../tex/figures/stream-summary-table-{i}.tex", "r") as f:
    lines = f.readlines()
    end = lines[-1]
    lines.pop(-1)
    lines.append("\\label{tbl:stream-summary}\n")
    lines.append(f"\\caption{{{caption}}}\n")
    lines.append(end)

with open(f"../tex/figures/stream-summary-table-{i}.tex", "w") as f:
    f.writelines(lines)

In [99]:
print(caption)

(1): \citet{odenkirchen:2009}, (2): \citet{grillmair:2009}, (3): \citet{koposov:2010}, (4): \citet{williams:2011}, (5): \citet{bonaca:2012}, (6): \citet{martin:2013}, (7): \citet{grillmair:2013}, (8): \citet{grillmair:2014}, (9): \citet{sesar:2015}, (10): \citet{kuzma:2015}, (11): \citet{bernard:2016}, (12): \citet{grillmair:2017}, (13): \citet{ibata:2017}, (14): \citet{grillmair:2017b}, (15): \citet{myeong:2017}, (16): \citet{shipp:2018}, (17): \citet{fu:2018}, (18): \citet{weiss:2018}, (19): \citet{mateu:2018}, (20): \citet{jethwa:2018}, (21): \citet{malhan:2018}, (22): \citet{ibata:2019}, (23): \citet{shipp:2019}, (24): \citet{malhan:2019}, (25): \citet{perottoni:2019}, (26): \citet{li:2019}, (27): \citet{ibata:2019b}, (28): \citet{caldwell:2020}, (29): \citet{ji:2020}, (30): \citet{shipp:2020}, (31): \citet{bonaca:2020}, (32): \citet{yuan:2020}, (33): \citet{sollima:2020}, (34): \citet{li:2021}, (35): \citet{gialluca:2021}, (36): \citet{sheffield:2021}, (37): \citet{ibata:2021}, (3

In [100]:
with open("../tex/refs.bib") as f:
    refs_text = f.read()


for cite_key, bibtex in bibcode_to_bib.values():
    if cite_key in refs_text:
        continue
    else:
        print(cite_key)
        # print(bibtex)